In [2]:
import pandas as pd
import numpy as np
import re
import requests
import time

from bs4 import BeautifulSoup

def moveSibling(tag, number):
	i = 1
	while i <= number:
		tag = tag.nextSibling
		i += 1
	return tag

from matplotlib import pyplot
%matplotlib inline

In [3]:
columns_dl = 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A'
downloaded_df = pd.DataFrame(data=None, columns=columns_dl)

In [4]:
for year in range(2006, 2017):
    y1 = str(year)[2:]
    y2 = str(year+1)[2:]
    url = "http://www.football-data.co.uk/mmz4281/" + y1 + y2 + "/D1.csv"
    df = pd.read_csv(url)
    df = df.loc[:, columns_dl]
    downloaded_df = downloaded_df.append(df)

In [5]:
seasonlist = [2006 + (i+1)/(9*34) for i in range(len(downloaded_df))]
gamedaylist = [1 + np.mod(i/9,34) for i in range(len(downloaded_df))]

In [6]:
downloaded_df.loc[:, "Season"] = seasonlist
downloaded_df.loc[:, "Gameday"] = gamedaylist

In [7]:
teamlist = downloaded_df.loc[:, "HomeTeam"].unique()
data = np.array([range(len(teamlist)), teamlist]).T
team_df = pd.DataFrame(data=data, columns=['TID', 'team'])

In [8]:
cleaned_df = pd.merge(team_df, downloaded_df, how='inner', left_on='team', right_on='HomeTeam')
cleaned_df = pd.merge(team_df, cleaned_df, how='inner', left_on='team', right_on='AwayTeam', suffixes=('_A', '_H'))

columns_cl = ['Season', 'Gameday', 'TID_H', 'TID_A', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']
cleaned_df = cleaned_df.loc[:, columns_cl].sort_values(['Season', 'Gameday'])
cleaned_df.index = range(len(cleaned_df))

cleaned_df.to_csv('Bundesliga_Data_2006_2016.csv', index=False)
team_df = team_df.set_index('TID')

In [ ]:
website = requests.get("http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/2015-16/30/2855445/spielanalyse_1-fsv-mainz-05-30_1-fc-koeln-16.html")
main = BeautifulSoup(website.content, 'html.parser')

In [ ]:
with open(str(year) + "_" + str(gameday) + "_main.html", 'w') as file:
    file.write((main.encode('ISO-8859-1')))

In [18]:
for year in range(2005, 2017):
    for gameday in range(1,35):
        print year, gameday
        url = "http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/"
        url = url + str(year) + "-" + str(year+1)[2:4] + "/"
        url = url + str(gameday) + "/0/spieltag.html"

        website = requests.get(url)
        main = BeautifulSoup(website.content, 'html.parser')
        main_table = main.find('table', {'class': 'tStat', 'summary': 'Tabelle'})

        # Once we get to the unplayed gamedays, no table will be displayed, and the loop shall end.
        try:
            main_standings = main_table.find_all('a', {'class': 'link verinsLinkBild'})
        except:
            break
            
        main_matches = main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})

        # Find all links of class "link." These refer to the game analyses.
        for game in main_matches.find_all('a', {'class' : 'link'}):
            # result = game.parent.previous_sibling.previous_sibling.text
            # result = map(int, result[0:result.find("(")-1].split(":"))

            game_url = game.get('href')
            encoded_in_url = game_url.split("/")

            # Game ID is the article ID for kicker, as unique ID for game
            game_id = encoded_in_url[8][0:7]

            # Game URL encodes the teams which are playing
            teams = encoded_in_url[9][13:-5].split("_")
            teams = map(int, [teams[i][teams[i].rfind('-')+1:] for i in range(2)])

            # URL to open individiual games
            game_url = "http://www.kicker.de" + game_url

            error_count = 0
            while error_count < 3:
                try:
                    game_file = requests.get(game_url)
                    game_soup = BeautifulSoup(game_file.content, 'html.parser')
                    filename = str(year) + "_"
                    filename += ("0" + str(gameday))[-2:] + "_" 
                    filename += str(teams[0]) + "-v-" + str(teams[1])
                    
                    with open("game_reports/" + filename + "_main.html", 'w') as file:
                        file.write(game_soup.encode('ISO-8859-1'))
                        break
                except:
                    error_count += 1

            if error_count == 3:
                continue

2005 1
2005 2
2005 3
2005 4
2005 5
2005 6
2005 7
2005 8
2005 9
2005 10
2005 11
2005 12
2005 13
2005 14
2005 15
2005 16
2005 17
2005 18
2005 19
2005 20
2005 21
2005 22
2005 23
2005 24
2005 25
2005 26
2005 27
2005 28
2005 29
2005 30
2005 31
2005 32
2005 33
2005 34
2006 1
2006 2
2006 3
2006 4
2006 5
2006 6
2006 7
2006 8
2006 9
2006 10
2006 11
2006 12
2006 13
2006 14
2006 15
2006 16
2006 17
2006 18
2006 19
2006 20
2006 21
2006 22
2006 23
2006 24
2006 25
2006 26
2006 27
2006 28
2006 29
2006 30
2006 31
2006 32
2006 33
2006 34
2007 1
2007 2
2007 3
2007 4
2007 5
2007 6
2007 7
2007 8
2007 9
2007 10
2007 11
2007 12
2007 13
2007 14
2007 15
2007 16
2007 17
2007 18
2007 19
2007 20
2007 21
2007 22
2007 23
2007 24
2007 25
2007 26
2007 27
2007 28
2007 29
2007 30
2007 31
2007 32
2007 33
2007 34
2008 1
2008 2
2008 3
2008 4
2008 5
2008 6
2008 7
2008 8
2008 9
2008 10
2008 11
2008 12
2008 13
2008 14
2008 15
2008 16
2008 17
2008 18
2008 19
2008 20
2008 21
2008 22
2008 23
2008 24
2008 25
2008 26
2008 27
2008

In [ ]:
try:
    kicker_grades_df = pd.read_csv('Grades_2005_2016.csv')
except:
    if input('Re-download? yes?') == 'yes':
        grade_row = []
        for year in range(2005, 2017):
            for gameday in range(1,35):
                print year, gameday
                url = "http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/"
                url = url + str(year) + "-" + str(year+1)[2:4] + "/"
                url = url + str(gameday) + "/0/spieltag.html"

                website = requests.get(url)
                main = BeautifulSoup(website.content, 'html.parser')
                main_table = main.find('table', {'class': 'tStat', 'summary': 'Tabelle'})

                # Once we get to the unplayed gamedays, no table will be displayed, and the loop shall end.
                try:
                    main_standings = main_table.find_all('a', {'class': 'link verinsLinkBild'})
                except:
                    break

                # If we didn't break, proceed as usual.

                main_matches = main.find('table', {'class': 'tStat tab1-bundesliga', 'summary': 'Begegnungen'})

                # Find all links of class "link." These refer to the game analyses.
                for game in main_matches.find_all('a', {'class' : 'link'}):
                    # result = game.parent.previous_sibling.previous_sibling.text
                    # result = map(int, result[0:result.find("(")-1].split(":"))

                    game_url = game.get('href')
                    encoded_in_url = game_url.split("/")

                    # Game ID is the article ID for kicker, as unique ID for game
                    game_id = encoded_in_url[8][0:7]

                    # Game URL encodes the teams which are playing
                    teams = encoded_in_url[9][13:-5].split("_")
                    teams = map(int, [teams[i][teams[i].rfind('-')+1:] for i in range(2)])

                    # URL to open individiual games
                    game_url = "http://www.kicker.de" + game_url
                    error_count = 0
                    while error_count < 10:
                        try:
                            game_file = requests.get(game_url)
                            game_soup = BeautifulSoup(game_file.content, 'html.parser')
                            game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'}).find_all('div', {'class': 'aufstellung_team'})
                            break
                        except:
                            error_count += 1

                    if error_count == 10:
                        continue

                    i = 0
                    for lineup in game_soup.find('table', {'class': 'tStat', 'summary': 'Vereinsliste'}).find_all('div', {'class': 'aufstellung_team'}):
                        no_players = 0
                        grade = 0.
                        for player in lineup.find_all('a'):
                            grade_raw = unicode(player.next_sibling)

                            grade_del1 = unicode(grade_raw).find('(')
                            grade_del2 = unicode(grade_raw).find(')')

                            if grade_del1 == -1:
                                continue

                            grade += float(grade_raw[grade_del1+1:grade_del2].replace(",", "."))

                            no_players += 1
                            if no_players == 11:
                                break
                        grade = grade/no_players

                        grade_row += [[year, gameday, teams[np.mod(i,2)], grade]]
                        i += 1
    else:
        "Doing nothing. No df loaded."

try:
    kicker_grades_df = kicker_grades_df.drop('Unnamed: 0', axis=1)
except:
    print "Import Successful"

kicker_grades_df.head()

In [ ]:
kicker_grades_df.to_csv('Grades_2005_2016.csv', index=False)

In [ ]:
team_list = []
for year in range(2005,2017):
    url = 'http://www.kicker.de/news/fussball/bundesliga/vereine/1-bundesliga/'
    url += str(year) + '-' + str(year+1)[-2:]
    url += '/vereine-liste.html'

    website = requests.get(url)
    main = BeautifulSoup(website.content, 'html.parser')
    for team in main.find_all('a', {'class': 'link verinsLinkBild'}):
        team_name_id = team.get('href').split('/')[7]
        team_name = team_name_id[0:team_name_id.rfind("-")]
        team_id = team_name_id[team_name_id.rfind("-")+1:]
        team_list += [[int(team_id), team_name]]
    

In [ ]:
kicker_team_df = pd.DataFrame(data=team_list, columns=['KID', 'KName'])
kicker_team_df = kicker_team_df.drop_duplicates().set_index('KID')

In [ ]:
key_df = pd.read_csv('Team_ID_hardcoded.csv')
key_df.head()

In [ ]:
def rescale_grade(x):
    return 1 - (x-1.)/5

grades_df = kicker_grades_df.merge(key_df, how='left', left_on='KID', right_on='KID').loc[:, ('Season', 'Gameday','TID', 'GradeAvg')]
grades_df = grades_df.set_index(['TID', 'Season', 'Gameday']).sort_index()
grades_df = grades_df.apply(rescale_grade)

print grades_df.loc[:, 'GradeAvg'].min()
print grades_df.loc[:, 'GradeAvg'].max()

In [ ]:
def inverse(x):
    return 1./x
def points(x):
    if x > 0:
        return 1
    elif x == 0:
        return 1./3
    else:
        return 0

In [ ]:
complete_df = cleaned_df.merge(grades_df, how='inner', left_on=['TID_H', 'Season', 'Gameday'], right_index=True)
complete_df = complete_df.merge(grades_df, how='inner', left_on=['TID_A', 'Season', 'Gameday'], right_index=True)

complete_df = complete_df.rename(columns={'GradeAvg_x': 'HGA', 'GradeAvg_y': 'AGA'})
complete_df = complete_df.replace({'H':1, 'D':0.5, 'A':0})
complete_df.loc[:, 'FTGD'] = complete_df.loc[:, 'FTHG'] - complete_df.loc[:, 'FTAG']
complete_df.loc[:, 'HTGD'] = complete_df.loc[:, 'HTHG'] - complete_df.loc[:, 'HTAG']

complete_df.loc[:, 'Odds'] = (complete_df.filter(regex='^B365').apply(inverse)*(1,0.5,0)).apply(np.sum, axis=1)

complete_df.head()

In [ ]:
tid=0

query = 'TID_H == ' + str(tid)
temp_H = complete_df.query(query).loc[:, ['TID_H', 'Season', 'Gameday', 'FTHG', 'FTAG']]
temp_H.loc[:, 'Gameday'] += 1
temp_H.loc[:, 'GD'] = temp_H.loc[:, 'FTHG'] - temp_H.loc[:, 'FTAG']
temp_H = temp_H.set_index(['TID_H', 'Season', 'Gameday'])

query = 'TID_A == ' + str(tid)
temp_A = complete_df.query(query).loc[:, ['TID_A', 'Season', 'Gameday', 'FTHG', 'FTAG']]
temp_A.loc[:, 'Gameday'] += 1
temp_A.loc[:, 'GD'] = - temp_A.loc[:, 'FTHG'] + temp_A.loc[:, 'FTAG']
temp_A = temp_A.set_index(['TID_A', 'Season', 'Gameday'])

temp = pd.concat([temp_H, temp_A]).sort_index()
temp.index = temp.index.rename(['TID', 'Season', 'Gameday'])
temp.loc[:, 'Points'] = temp.loc[:, 'GD'].apply(points).to_frame()

temp = temp.drop(['FTHG', 'FTAG'], axis=1)

table_df = temp.groupby(level=['TID', 'Season'], group_keys=False).cumsum()


In [ ]:
multi_index = pd.MultiIndex(levels=[[],[],[]],
                             labels=[[],[],[]],
                             names=[u'TID', u'Season', u'Gameday'])
running_df = pd.DataFrame(data=None, columns=['Points'], index=multi_index)

for tid in range(len(key_df)):
    query = 'TID_H == ' + str(tid)
    temp_H = complete_df.query(query).loc[:, ['TID_H', 'Season', 'Gameday', 'FTHG', 'FTAG']]
    temp_H.loc[:, 'Gameday'] += 1
    temp_H.loc[:, 'GD'] = temp_H.loc[:, 'FTHG'] - temp_H.loc[:, 'FTAG']
    temp_H = temp_H.set_index(['TID_H', 'Season', 'Gameday'])

    query = 'TID_A == ' + str(tid)
    temp_A = complete_df.query(query).loc[:, ['TID_A', 'Season', 'Gameday', 'FTHG', 'FTAG']]
    temp_A.loc[:, 'Gameday'] += 1
    temp_A.loc[:, 'GD'] = - temp_A.loc[:, 'FTHG'] + temp_A.loc[:, 'FTAG']
    temp_A = temp_A.set_index(['TID_A', 'Season', 'Gameday'])

    temp = pd.concat([temp_H, temp_A]).sort_index()
    temp.index = temp.index.rename(['TID', 'Season', 'Gameday'])
    temp = temp.loc[:, 'GD'].apply(points).rename('Points').to_frame()
    
    running_df = running_df.append(temp.groupby(level=['TID', 'Season'], group_keys=False).rolling(3, 1).mean())
   

In [ ]:
running_df.to_csv('Running_Points_2006_2013.csv')
len(running_df)/2

In [ ]:
complete_running_df = complete_df.merge(running_df, how='left', left_on=['TID_H', 'Season', 'Gameday'], right_index=True)
complete_running_df = complete_running_df.merge(running_df, how='left', left_on=['TID_A', 'Season', 'Gameday'], right_index=True)

complete_running_df = complete_running_df.rename(columns={'Points_x': 'HP3', 'Points_y': 'AP3'}).filter(regex='^(?!B365)')
complete_running_df = complete_running_df.drop(['FTAG', 'FTR','HTAG', 'HTR'], axis=1)

complete_running_df = complete_running_df.drop_duplicates(['Season', 'Gameday', 'TID_H', 'TID_A']).fillna(0.5)

In [ ]:
complete_running_df.to_csv('All_Data_2006_2016.csv')
len(complete_running_df)

In [ ]:
complete_running_df.fillna(0.5)

In [ ]:
print complete_running_df.loc[:, ['HST', 'AST', 'HS', 'AS']].mean()
print 5.57/14.61
print 4.49/11.82

In [ ]:
shots = complete_running_df.loc[:, ['TID_H','Season', 'HST', 'AST', 'HS', 'AS']].groupby(['TID_H', 'Season']).mean()
conversion_df = shots
conversion_df.loc[:, 'HEff'] = np.round(100*conversion_df.loc[:, 'HST'] / conversion_df.loc[:, 'HS'], 1)
conversion_df.loc[:, 'AEff'] = np.round(100*conversion_df.loc[:, 'AST'] / conversion_df.loc[:, 'AS'], 1)

print conversion_df.loc[:, ['HEff', 'AEff']].mean()
#conversion_df.index = conversion_df.set_index(range(len(conversion_df)))


In [ ]:
team_df

In [ ]:
cutoff_GD = 2
print complete_running_df.query('TID_H == 30').loc[:, 'FTGD'].apply(min, args=(cutoff_GD,)).apply(max, args=(-cutoff_GD,)).value_counts(False, False, True)
print (-complete_running_df.query('TID_A == 30').loc[:, 'FTGD'].apply(min, args=(cutoff_GD,)).apply(max, args=(-cutoff_GD,))).value_counts(False, False, True)

In [ ]:
shots = complete_running_df.loc[:, ['TID_A', 'HST', 'AST', 'HS', 'AS']].groupby('TID_A').mean()
conversion_df = shots.merge(team_df, left_index=True, right_index=True).set_index('team')
conversion_df.loc[:, 'HEff'] = np.round(100*conversion_df.loc[:, 'HST'] / conversion_df.loc[:, 'HS'], 1)
conversion_df.loc[:, 'AEff'] = np.round(100*conversion_df.loc[:, 'AST'] / conversion_df.loc[:, 'AS'], 1)

print conversion_df.loc[:, ['HEff', 'AEff']].mean()
conversion_df

In [ ]:
complete_df.plot(kind='scatter', x='HGA', y='FTGD')
complete_df.plot(kind='scatter', x='AGA', y='FTGD')
complete_running_df.plot.hexbin(x='HGA', y='AGA', gridsize=20)
complete_running_df.plot.hexbin(x='HGA', y='AGA', C='Odds', gridsize=20)

In [ ]:
complete_running_df.plot(kind='scatter', x='HP3', y='AP3')

In [ ]:
complete_running_df.loc[:, ['Season', 'HGA', 'AGA']].groupby('Season').mean().plot()
complete_running_df.loc[:, ['Season', 'FTGD', 'HTGD']].groupby('Season').mean().plot()
(complete_running_df.loc[:, ['Season', 'FTGD', 'HTGD']].groupby('Season').std()*(1,np.sqrt(2.))).plot()

In [ ]:
complete_df.plot.hexbin(x='Odds', y='FTGD', gridsize=15)